In [77]:
import numpy as np
import pandas as pd
import os, random, shutil, sys, time
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sys

#mac
util_path = "/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/util"

#constance
'''args = sys.argv[0:]
py_file = args[0]

code_location = os.path.dirname(os.path.abspath(py_file))
index_of_latest = code_location.index('latest')
util_path = os.path.join(code_location[:index_of_latest], "latest", "util")
'''

sys.path.insert(0, util_path)

In [78]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/util


In [79]:
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/Brianna/tonalide/min_value_from_10_uM/tonalide_min_value_from_10_uM.csv'

df_test_dose_response = pd.read_csv(complete_file_path, header = 0)
pd.set_option('display.max_columns', None)
display(df_test_dose_response.head())
#display(np.unique(morpho_data.well))

,dose,num_affected,total_num
0,0,1.519964,100
1,10,0.000000,100
2,25,6.893015,100
3,50,26.519651,100
4,75,36.724537,100


In [80]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [81]:
# Fit dose response models
start_time = time.time()
model_predictions = bmdest.analyze_dose_response_data(df_test_dose_response)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("BMD calculation is done, it took:"+str(time_took)) 

BMD calculation is done, it took:0.8 seconds


In [82]:
print (model_predictions)

            Model  Chi-squared         p-val         AIC      BMD10  \
0        logistic    18.761566  2.128891e-03  512.760934  38.873198   
1           gamma     4.589215  3.320996e-01  499.368034  33.543824   
2         weibull          NaN           NaN         NaN        NaN   
3    log_logistic     7.458424  1.135567e-01  502.321889  35.911090   
4          probit    61.790958  5.180599e-12  547.682339  50.745654   
5      log_probit          NaN           NaN         NaN        NaN   
6    multistage_2    19.857226  5.328723e-04  523.605762  24.319125   
7  quantal_linear    40.199083  1.361487e-07  546.126188  13.218896   

      BMDL10       BMD50                                   Scaled Residuals  \
0  35.014587   76.321085  [1.094899835808857, 2.300913928665122, 0.53910...   
1  28.422500   84.976016  [-0.7384753810819761, 1.191298226945695, -0.35...   
2        NaN         NaN                [nan, nan, nan, nan, nan, nan, nan]   
3  30.713144   82.034986  [-0.56164453032130

In [83]:
# Select best model
selected_model_params = bmdest.select_model(model_predictions)

In [84]:
chemical_id = "tonalide"
end_point = "CTG"
bmd_feasibility_flag = 2


# Check if unique model is found
unique_model_flag = selected_model_params['no_unique_model_found_flag']
if(unique_model_flag == 0):
    # Generate report
    #print(df_test_dose_response.dose[-1:])

    ps.save_results_good_data_unique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
else:
    bmd_analysis_flag = selected_model_params['model_select_flag']
    if(bmd_analysis_flag == 1):
        ps.save_results_poor_data_or_no_convergence(df_test_dose_response, bmd_feasibility_flag, str(chemical_id), end_point, selected_model_params)
    else:
        ps.save_results_good_data_nounique_model(df_test_dose_response, bmd_feasibility_flag, model_predictions, selected_model_params, str(chemical_id), end_point)